![CH6-ADS.png](.\Media\CH6-ADS.png)
# <span style="color:#cc5500;">Routine SQL Maintenance </span>



# <span style="color:#cc5500;">Introduction</span>

The purpose of this notebook is to provide database administrators (DBAs) with a clear and concise guide to the tasks and settings necessary to keep SQL Server 2016+ instances running safely and efficiently. The scripts and schedules in this guide are intended to be a starting place for maintenance. Each organization and application is different, so changes in practice to the setting and schedules in this document are expected.

<span style="color:#5FB404;">At the top of your screen, in the menu bar above, select SQL as the Kernel, and Attach to: (select your SQL Instance)&nbsp; When Prompted, Authenticate to your Instance and in the Drop-down, select your target database.</span>

## <span style="color:#cc5500;">Maintenance</span>

This section outlines the tasks needed to operate and manage a Microsoft SQL Server. Further information on these tasks can be found in later sections of this document as linked.

### <span style="color:#cc5500;">Index Maintenance</span>

Index maintenance can be automated using the Maintenance Plan Wizard in SQL Server Management Studio.  Indexes in SQL Server can become fragmented over time which can affect performance.  Reorganizing an index can reduce its fragmentation, can be interrupted at any time without a loss of work already performed, and can be run during production with almost no effect on performance.  Rebuilding indexes will eliminate fragmentation in an index.  This process must complete in order to affect the performance of the database server.  The rebuild process is resource intensive, so rebuilding indexes should only be performed during maintenance windows.  It is common to reorganize indexes on a nightly or weekly basis and rebuild indexes on a weekly or monthly basis.  How often indexes should be reorganized and rebuilt depends on how quickly the indexes become fragmented.  The next section will describe one way to monitor for fragmentation which will help the DBA in deciding which indexes and databases should be maintained and the schedule for that maintenance.  You can read up on Index Maintenance here: [Maintenance Plans - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/maintenance-plans/maintenance-plans?view=sql-server-2017)

<span style="color:#5FB404;">Click in the Code block below, click the edit control and replace 'Adventureworks' with your database name, and 'HumanResources.Employee' with your table.</span>

<span style="color:#5FB404;">Then click the Run Cell button. To avoid repitition, it is assumed that you will modify the appropriate codeblocs for the remainder of this notebook</span>

The Dynamic Management View dm.db_index_physical_stats returns, among other things, avg_fragmentation_in_percent. This field is the logical fragmentation for indexes, or extent fragmentation for heaps in the IN_ROW_DATA allocation unit.

In [ ]:
SELECT object_id, index_id, avg_fragmentation_in_percent, page_count
FROM sys.dm_db_index_physical_stats(DB_ID('AdventureWorks'), OBJECT_ID('HumanResources.Employee'), NULL, NULL, NULL);

The easiest way to maintain database indexes is to include daily reorganizations for all indexes and weekly rebuilding of indexes in a Maintenance Plan. Instructions for setting up a maintenance plan are located here: [Create a Maintenance Plan - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/maintenance-plans/create-a-maintenance-plan?view=sql-server-2017)

### <span style="color:#cc5500;">Statistics</span>

The Query Optimizer uses statistics to create query plans that improve query performance.  For most queries, the Query Optimizer already generates the necessary statistics for a high quality query plan; in some cases, you need to create additional statistics or modify the query design for best results.  Discussed later in the Configuration Guide section, enabling Auto Update Statistics and Auto Create Statistics alleviates most, if not all, of the need to manually create statistics.  As mentioned above, statistics are used by the Query Optimizer to help choose the best execution plan, including what indexes to use, when compiling a query.  When you perform index rebuild maintenance, any statistics associated with the index itself will be updated; however, any statistics associated with a column that is not included in an index will not be updated.  With Auto Update Statistics enabled and for SQL Server versions 2012 through 2014, the threshold for auto update of statistics requires 20% of rows in a table to change. Therefore, you should include an update statistics task in your maintenance plan. Updating statistics regularly can help further reduce the need for full index rebuild operations: [Statistics - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/statistics/statistics?view=sql-server-2017) and [Default auto statistics update threshold change for SQL Server 2016 | Microsoft Docs](https://docs.microsoft.com/en-us/archive/blogs/psssql/default-auto-statistics-update-threshold-change-for-sql-server-2016)

## <span style="color:#cc5500;">Monitoring SQL Server</span>

This section outlines the tasks related to monitoring your SQL Server. Further information on these tasks can be found in later sections of this document as linked.

### <span style="color:#cc5500;">Review the SQL Server Error Log</span>

<span style="font-size:11.0pt;font-family:&quot;Segoe UI&quot;,sans-serif;color:#E3E3E3;">View the SQL Server error log to ensure that processes have completed successfully (for example, backup and restore operations, batch commands, or other scripts and processes). This can be helpful to detect any current or potential problem areas, including automatic recovery messages (particularly if an instance of SQL Server has been stopped and restarted), kernel messages, or other server-level error messages.</span> <span style="font-size:11.0pt;font-family:&quot;Segoe UI&quot;,sans-serif;color:#E3E3E3;">View the SQL Server error log by using SQL Server Management Studio or any text editor.&nbsp; For more information about how to view the error log, see&nbsp;<a href="https://docs.microsoft.com/en-us/sql/relational-databases/logs/open-log-file-viewer?view=sql-server-ver15" style="cursor:pointer;overflow-wrap:break-word;outline-width:0px;" rel="nofollow">Open Log File Viewer</a>. By default, the error log is located at&nbsp;<code style="border-radius:3px;overflow-wrap:break-word;"><span style="font-family:&quot;Segoe UI&quot;,sans-serif;">Program Files\Microsoft SQL Server\MSSQL.</span></code><em>n</em><code style="border-radius:3px;overflow-wrap:break-word;"><span style="font-family:&quot;Segoe UI&quot;,sans-serif;">\MSSQL\LOG\ERRORLOG</span></code>&nbsp;and&nbsp;<code style="border-radius:3px;overflow-wrap:break-word;"><span style="font-family:&quot;Segoe UI&quot;,sans-serif;">ERRORLOG.</span></code><em>n</em>&nbsp;files.</span>  <span style="font-size:11.0pt;font-family:&quot;Segoe UI&quot;,sans-serif;color:#E3E3E3;">A new error log is created each time an instance of SQL Server is started, although the&nbsp;<a href="https://docs.microsoft.com/en-us/sql/relational-databases/system-stored-procedures/sp-cycle-errorlog-transact-sql?view=sql-server-ver15" style="cursor:pointer;overflow-wrap:break-word;outline-width:0px;" rel="nofollow">sp_cycle_errorlog</a>&nbsp;system stored procedure can be used to cycle the error log files without having to restart the instance of SQL Server. Typically, SQL Server retains backups of the previous six logs and gives the most recent log backup the extension .1, the second most recent the extension .2, and so on. The current error log has no extension.</span> <span style="font-size:11.0pt;font-family:&quot;Segoe UI&quot;,sans-serif;color:#E3E3E3;">Be aware that you can also view the SQL Server error log on instances of SQL Server that are offline or cannot start. For more information, see&nbsp;<a href="https://docs.microsoft.com/en-us/sql/relational-databases/logs/view-offline-log-files?view=sql-server-ver15" style="cursor:pointer;overflow-wrap:break-word;outline-width:0px;" rel="nofollow">View Offline Log Files</a>.</span>

### <span style="color:#cc5500;">Review the SQL Job History</span>

SQL Server utilizes the MSDB database in conjunction with the SQL Server Agent Service to execute and track jobs. These jobs perform functions such as executing maintenance plans described in the previous section Maintenance Guide. It is important to review the job history for success (or failure) of SQL Server jobs to ensure that maintenance and other automated tasks are functioning properly. The easiest way to review job history is by using SQL Server Management Studio:

1. In Object Explorer, connect to an instance of the SQL Server Database Engine, and then expand that instance.
2. Expand SQL Server Agent, and then expand Jobs.
3. Right-click a job, and then click View History.
4. In the Log File Viewer, view the job history.
5. To update the job history, click Refresh. To view fewer rows, click the Filter button and enter filter parameters

In [ ]:
-- lists all job information for the NightlyBackups job.  Mofidify this for your 'name'  
USE msdb ;  
GO  

EXEC dbo.sp_help_jobhistory   
    @job_name = N'NightlyBackups' ;  
GO

### <span style="color:#cc5500;">Running DBCC CheckDB</span>

DBCC CHECKDB checks the logical and physical integrity of all objects in the specified database. All the DBCC validation commands use database snapshots to keep validation operations from interfering with on-going database operations and to allow the validation operation to see a quiescent, consistent view of the data. This snapshot can be as large as the database being checked, so make sure you have enough disk space available to accommodate the database snapshot

<span style="font-family:&quot;Segoe UI&quot;,sans-serif;color:#E3E3E3;">There is a completely dedicated chapter in this notebook on DBCC.&nbsp; See Chapter 5 Database Corruptions.</span>

<span style="font-family:&quot;Segoe UI&quot;,sans-serif;color:#E3E3E3;">Checks the logical and physical integrity of all the objects in the specified database by performing the following operations:</span>

- <span style="font-family:&quot;Segoe UI&quot;,sans-serif;color:#E3E3E3;">Runs&nbsp;<a href="https://docs.microsoft.com/en-us/sql/t-sql/database-console-commands/dbcc-checkalloc-transact-sql?view=sql-server-2017" rel="nofollow"><span style="color:blue;">DBCC CHECKALLOC</span></a>&nbsp;on the database.</span>
- Runs [<span style="color:blue;">DBCC CHECKTABLE</span>](https://docs.microsoft.com/en-us/sql/t-sql/database-console-commands/dbcc-checktable-transact-sql?view=sql-server-2017) <span style="color:rgb(227, 227, 227);font-family:&quot;Segoe UI&quot;, sans-serif;">&nbsp;on every table and view in the database.</span>
- Runs [<span style="color:blue;">DBCC CHECKCATALOG</span>](https://docs.microsoft.com/en-us/sql/t-sql/database-console-commands/dbcc-checkcatalog-transact-sql?view=sql-server-2017) <span style="color:rgb(227, 227, 227);font-family:&quot;Segoe UI&quot;, sans-serif;">&nbsp;on the database.</span>
- Validates the contents of every indexed view in the database.
- Validates link-level consistency between table metadata and file system directories and files when storing **varbinary(max)** <span style="color:rgb(227, 227, 227);font-family:&quot;Segoe UI&quot;, sans-serif;">&nbsp;data in the file system using FILESTREAM.</span>
- Validates the Service Broker data in the database.

Read up on DBCC here: [DBCC CHECKDB (Transact-SQL) - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/t-sql/database-console-commands/dbcc-checkdb-transact-sql?view=sql-server-2017)

In [ ]:
-- Check the current database.    
DBCC CHECKDB;    
GO    
-- Check the AdventureWorks2012 database without nonclustered indexes.    
DBCC CHECKDB (AdventureWorks2012, NOINDEX);    
GO

The best way to run DBCC CheckDB on a consistent basis is to enable a step in a maintenance plan to check the consistency of the database.  It is best to run the DBCC CheckDB maintenance task prior to performing the week’s full backup. Any consistency errors will be recorded in the maintenance task’s error log and help provide confidence in the integrity of the subsequent full database backup.

DBCC CheckDB should also be run after production databases are restored to test servers.  This will test both the production database and the backup and restore system for errors,

<span style="color:#cc5500;">Review the Cluster Log</span>

In clustered SQL Server environments, you might have availability problems or unexpected failovers causing you headaches. Factors external to SQL Server can contribute to these types of problems. Generating a set of cluster logs can help you quickly identify problems happening at the cluster level.

The following PowerShell command, executed from one of the nodes of your SQL Server cluster will produce a cluster log set in the directory of your choosing:

Here is a list of the PowerShell Cmlets related to the Cluster Log: [Get-ClusterLog (FailoverClusters) | Microsoft Docs](https://docs.microsoft.com/en-us/powershell/module/failoverclusters/get-clusterlog?view=windowsserver2019-ps&viewFallbackFrom=win10-ps)  Heres a good article on how to use PowerShell to find Cluster Log Errors: [Finding Cluster Log Errors with Powershell | John Morehouse (sqlrus.com)](https://sqlrus.com/2019/01/finding-cluster-log-errors/)

In [ ]:
-- Change the Kernel from TSQL to PowerShell to run this
Get-ClusterLog -D C:\temp\clusterLogs

## <span style="color:#cc5500;">Auto Update Statistics, Auto Create Statistics and Auto Shrink</span>

The Query Optimizer uses statistics to create query plans that improve query performance.  For most queries, the Query Optimizer already generates the necessary statistics for a high-quality query plan; in some cases, you need to create additional statistics or modify the query design for best results.  This section discusses statistics concepts and provides guidelines for using query optimization statistics effectively.

Components and concepts Statistics Statistics for query optimization are binary large objects (BLOBs) that contain statistical information about the distribution of values in one or more columns of a table or indexed view.  The Query Optimizer uses these statistics to estimate the cardinality, or number of rows, in the query result.  These cardinality estimates enable the Query Optimizer to create a high-quality query plan.  For example, depending on your predicates, the Query Optimizer could use cardinality estimates to choose the index seek operator instead of the more resource-intensive index scan operator, if doing so improves query performance.

Each statistics object is created on a list of one or more table columns and includes a histogram displaying the distribution of values in the first column.  Statistics objects on multiple columns also store statistical information about the correlation of values among the columns. These correlation statistics, or densities, are derived from the number of distinct rows of column values.

### <span style="color:#cc5500;">Histogram</span>

A histogram measures the frequency of occurrence for each distinct value in a data set. The Query Optimizer computes a histogram on the column values in the first key column of the statistics object, selecting the column values by statistically sampling the rows or by performing a full scan of all rows in the table or view.  If the histogram is created from a sampled set of rows, the stored totals for number of rows and number of distinct values are estimates and do not need to be whole integers.

To create the histogram, the Query Optimizer sorts the column values, computes the number of values that match each distinct column value, and then aggregates the column values into a maximum of 200 contiguous histogram steps. Each histogram step includes a range of column values followed by an upper bound column value. The range includes all possible column values between boundary values, excluding the boundary values themselves. The lowest of the sorted column values is the upper boundary value for the first histogram step.

In more detail, SQL Server creates the histogram from the sorted set of column values in three steps:  

- Histogram initialization: In the first step, a sequence of values starting at the beginning of the sorted set is processed, and up to 200 values of range\_high\_key, equal\_rows, range\_rows, and distinct\_range\_rows are collected (range\_rows and distinct\_range\_rows are always zero during this step). The first step ends either when all input has been exhausted, or when 200 values have been found.
- Scan with bucket merge: Each additional value from the leading column of the statistics key is processed in the second step, in sorted order; each successive value is either added to the last range or a new range at the end is created (this is possible because the input values are sorted). If a new range is created, then one pair of existing, neighboring ranges is collapsed into a single range. This pair of ranges is selected in order to minimize information loss. This method uses a maximum difference algorithm to minimize the number of steps in the histogram while maximizing the difference between the boundary values. The number of steps after collapsing ranges stays at 200 throughout this step.
- Histogram consolidation: In the third step, more ranges may be collapsed if a significant amount of information is not lost. The number of histogram steps can be fewer than the number of distinct values, even for columns with fewer than 200 boundary points. Therefore, even if the column has more than 200 unique values, the histogram may have fewer than 200 steps. For a column consisting of only unique values, then the consolidated histogram will have a minimum of three steps.

AUTO\_CREATE\_STATISTICS option

  

When the automatic create statistics option, AUTO\_CREATE\_STATISTICS is ON, the Query Optimizer creates statistics on individual columns in the query predicate, as necessary, to improve cardinality estimates for the query plan. These single-column statistics are created on columns that don't already have a histogram in an existing statistics object. The AUTO\_CREATE\_STATISTICS option does not determine whether statistics get created for indexes. This option also does not generate filtered statistics. It applies strictly to single-column statistics for the full table.

  

When the Query Optimizer creates statistics as a result of using the AUTO\_CREATE\_STATISTICS option, the statistics name starts with \_WA. You can use the following query to determine if the Query Optimizer has created statistics for a query predicate column.

In [ ]:
SELECT OBJECT_NAME(s.object_id) AS object_name,  
    COL_NAME(sc.object_id, sc.column_id) AS column_name,  
    s.name AS statistics_name  
FROM sys.stats AS s 
INNER JOIN sys.stats_columns AS sc  
    ON s.stats_id = sc.stats_id AND s.object_id = sc.object_id  
WHERE s.name like '_WA%'  
ORDER BY s.name;

AUTO\_UPDATE\_STATISTICS option

  

When the automatic update statistics option, AUTO\_UPDATE\_STATISTICS is ON, the Query Optimizer determines when statistics might be out-of-date and then updates them when they are used by a query. This action is also known as statistics recompilation. Statistics become out-of-date after modifications from insert, update, delete, or merge operations change the data distribution in the table or indexed view. The Query Optimizer determines when statistics might be out-of-date by counting the number of row modifications since the last statistics update and comparing the number of row modifications to a threshold. The threshold is based on the table cardinality, which can be defined as the number of rows in the table or indexed view.

In [ ]:
--  TSQL Script to show your database properties with respect to statistics and auto-shrink
Use master
GO
SELECT name as Name,database_id as Database_ID,compatibility_level as Compat_Lvl,
CASE is_auto_shrink_on WHEN 1 THEN 'YES - Please Fix' ELSE 'No' END as Is_Auto_Shrink_On,
recovery_model_desc as Recovery_Model,
CASE page_verify_option_desc WHEN 'CHECKSUM' THEN 'Yes' ELSE page_verify_option_desc END as Is_Page_Verify_Option_CheckSum,
CASE is_auto_create_stats_on WHEN 1 THEN 'Yes' ELSE 'NO - Please Fix' END as Is_Auto_Create_Stats_On,
CASE is_auto_update_stats_on WHEN 1 THEN 'Yes' ELSE 'NO - Please Fix' END as Is_Auto_Update_Stats_On
from sys.databases

### <span style="color:#cc5500;">Considerations for AUTO_SHRINK</span>

As a general rule-of-thumb, I usually do not recommend using the AUTO\_SHRINK feature accept in certain cases.

AUTO\_SHRINK is a database option in SQL Server.  When you enable this option for a database, this database becomes eligible for shrinking by a background task.  This background task evaluates all databases that satisfy the criteria for Shrinking and shrink the data or log files.

You have to carefully evaluate setting this option for the databases in a SQL Server instance. Frequent grow and shrink operations can lead to various performance problems.

1. If multiple databases undergo frequent shrink and grow operations, then this will easily lead to file system level fragmentation.  This can have a severe impact on performance.  This is true whether you use the automatic settings or whether you manually grow and shrink the files frequently
2. After AUTO\_SHRINK successfully shrinks the data or log file, a subsequent DML or DDL operation can slow down significantly if space is required and the files need to grow.
3. The AUTO\_SHRINK background task can take up resources when there are many databases that need shrinking.
4. The AUTO\_SHRINK background task will need to acquire locks and other synchronization that can conflict with other regular application activity.

Consider setting databases to a required size and pre-grow them.  Leave the unused space in the database files if you think the application usage patterns will need them again.  This can prevent frequent shrink and growth of the database files.

### <span style="color:#cc5500;">Considerations for AUTOGROW</span>

- If you run a transaction that requires more log space than is available, and you have turned on the autogrow option for the transaction log of that database, then the time it takes the transaction to complete will include the time it takes the transaction log to grow by the configured amount.  If the growth increment is large or there is some other factor that causes it to take a long time, the query in which you open the transaction might fail because of a timeout error.  The same sort of issue can result from an autogrow of the data portion of your database.
- If you run a large transaction that requires the log to grow, other transactions that require a write to the transaction log will also have to wait until the grow operation completes.
- If you have many file growths in your log files, you may have an excessively large number of virtual log files (VLF). This can lead to performance problems with database startup/online operations, replication, mirroring, and change data capture (CDC). Additionally, this can sometimes cause performance problems with data modifications.

### <span style="color:#cc5500;">Best Practices for Autogrow and Autoshrink</span>

- For a managed production system, you must consider autogrow to be merely a contingency for unexpected growth.  Do not manage your data and log growth on a day-to-day basis with autogrow.
- You can use alerts or monitoring programs to monitor file sizes and grow files proactively.  This helps you avoid fragmentation and permits you to shift these maintenance activities to non-peak hours.
- Auto-Shrink and autogrow must be carefully evaluated by a trained Database Administrator (DBA); They must not be left unmanaged.
- Your autogrow increment must be large enough to avoid the performance penalties listed in the previous section.  The exact value to use in your configuration setting and the choice between a percentage growth and a specific MB size growth depends on many factors in your environment. A general rule of thumb to you can use for testing is to set your autogrow setting to about one-eight the size of the file.
- Turn on the setting for each file to prevent any one file from growing to a point where it uses up all available disk space.
- Keep the size of your transactions as small as possible to prevent unplanned file growth.

### <span style="color:#cc5500;">Why do I have to worry about disk space if size settings are automatically controlled?</span>

- The autogrow setting cannot grow the database size beyond the limits of the available disk space on the drives for which files are defined. Therefore, if you rely on the autogrow functionality to size your databases, you must still independently check your available hard disk space. The autogrow setting is also limited by the MAXSIZE parameter you select for each file. To reduce the possibility of running out of space, you can monitor the Performance Monitor counter SQL Server: Databases Object: Data File(s) Size (KB) and set up an alert for when the database reaches a certain size.
- Unplanned growth of data or log files can take space that other applications expect to be available and might cause those other applications to experience problems.
- The growth increment of your transaction log must be large enough to stay ahead of the needs of your transaction units. Even with autogrow turned on, you can receive a message that the transaction log is full, if it cannot grow fast enough to satisfy the needs of your query.
- SQL Server does not constantly test for databases that have hit the configured threshold for autoshrink. Instead, it looks at the available databases and finds the first one that is configured to autoshrink. It checks that database and shrinks that database if needed. Then, it waits several minutes before checking the next database that is configured for autoshrink. In other words, SQL Server does not check all databases at once and shrink them all at once. It will work through the databases in a round robin fashion to stagger out the load over a period of time. Therefore, depending on how many databases on a particular SQL Server instance you have configured to autoshrink, it might take several hours from the time the database hits the threshold until it actually shrinks.

  

Helpful references:

  

[Troubleshoot full transaction log error 9002 - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/logs/troubleshoot-a-full-transaction-log-sql-server-error-9002?view=sql-server-ver15)  

[Database Instant File Initialization - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/databases/database-instant-file-initialization?view=sql-server-ver15)  

[Transaction Log Architecture and Management Guide - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/sql-server-transaction-log-architecture-and-management-guide?view=sql-server-ver15)  

[Manage Transaction Log File Size - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/logs/manage-the-size-of-the-transaction-log-file?view=sql-server-ver15)